# CALIPSO Ozone Number Density

### Summary

The goal of this notebook is to show how to plot vertical distribution of ozone number density (m^-3) using the `pyhdf` library for a latitudinal section. The data are from the CALIPSO Lidar Level 1B profile data, V4-51.

### Prerequisites

This notebook was written using Python 3.8.10, and requires these libraries and files:
- [numpy](https://numpy.org/)
- [matplotlib](http://matplotlib.org/)
- [pyhdf](https://github.com/fhs/pyhdf)
- [os](https://docs.python.org/3/library/os.html)
- [earthacess](https://earthaccess.readthedocs.io/en/latest/)

### Notebook Author / Affiliation
Cheyenne Land / Atmospheric Science Data Center

## 1. Setup

In [ ]:
from pyhdf.SD import SD, SDC
import numpy as np
from matplotlib import pyplot as PLT
import earthaccess
import os

%reload_ext autoreload
%autoreload 2

## 2. Search for data using earthaccess

In [ ]:
earthaccess.login()
short_name = 'CAL_LID_L1-Standard-V4-51'
version = 'V4-51'
results = earthaccess.search_data(
    short_name= short_name,
    version = version,
    temporal=("2020-02-01T00:00:00", "2020-02-01T01:59:59")
)
print(f'{len(results[:])} file(s) found.')
print(results)

## 3. Download data

In [ ]:
file_names = []
cwd = os.getcwd()
for r in results[:]:
    file_name = r['umm']['DataGranule']['Identifiers'][0]['Identifier']
    earthaccess.download(r, cwd)
    print(f'File downloaded: {file_name}')   
    file_names.append(file_name)

        

## 4. Generate Plot

Ozone_number_density [# of single shot (333m) resn profiles in file (55725), met_data_altitude (33)]

In [ ]:
#Open and read file
for file in file_names:
    data_file = os.path.join(f'{cwd}', file)
    hdf = SD(data_file, SDC.READ)
    ds = hdf.datasets()

    #retrieve data
    hdfdataobject = hdf.select('Ozone_Number_Density')
    latitude = hdf.select('Latitude')
    data = hdfdataobject.get()
    ozone_number_density = np.transpose(data)
    lat = latitude.get()[:,0]

    #Interval for altitude to plot the y-axis
    #Range is from -2 km - 40 km of 33 range bins. For more detail go to CALIPSO Data Products Catalog at:
    # https://www-calipso.larc.nasa.gov/products/CALIPSO_DPC_Rev4x93.pdf
    met_data_altitudes = np.linspace(-2,40,33)

    #plot data
    PLT.figure(figsize=(7.20,3.60))
    im = PLT.contourf(lat, met_data_altitudes, ozone_number_density,
                    cmap=PLT.get_cmap('jet'))
    #collection_name = data_file.split('/')[-1]
    PLT.title(f'{file}\n Ozone Number Density',
    fontsize=8)
    PLT.ylabel("Altitude (km)", fontsize = 8)
    PLT.xlabel("Latitude (deg)", fontsize = 8)
    cb = PLT.colorbar(im , shrink=0.90)
    cb.set_label('Ozone Number Denisty m-³', fontsize=8)
    PLT.savefig(f'{file}_Ozone_Number_Density.jpg', dpi=200)
    print(f'Plot has been saved: {cwd}/{file}_Ozone_Number_Density.jpg')
    print('Removing data file...')
    os.remove(data_file)